In [6]:
import pandas as pd;

#Load dataset and examine dataset, rename columns to questions and answers

df=pd.read_excel("WHO_FAQ.xlsx")
df.columns=["context","answers"]

print(df)

                                              context                                            answers
0                              What is a coronavirus?  Coronaviruses are a large family of viruses wh...
1                                   What is COVID-19?  COVID-19 is the infectious disease caused by t...
2                  What are the symptoms of COVID-19?  The most common symptoms of COVID-19 are fever...
3                           How does COVID-19 spread?  People can catch COVID-19 from others who have...
4   Can the virus that causes COVID-19 be transmit...  Studies to date suggest that the virus that ca...
5   Can CoVID-19 be caught from a person who has n...  The main way the disease spreads is through re...
6   Can I catch COVID-19 from the feces of someone...  The risk of catching COVID-19 from the feces o...
7   What can I do to protect myself andProtection ...  Self-isolate by staying at home if you begin t...
8   Protection measures for persons who are in or ...  

In [7]:
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word) 
    #sentence=sent_stemmed
    
    return sentence
                    
def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["context"]]
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["context"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['coronavirus', 'covid19', 'symptoms covid19', 'covid19 spread', 'virus causes covid19 transmitted air', 'covid19 caught person symptoms', 'catch covid19 feces disease', 'protect andprotection measures persons recently visited past 14 days areas covid19 spreading prevent spread disease', 'protection measures persons recently visited past 14 days areas covid19 spreading', 'likely catch covid19', 'likely catch covid19', 'worry covid19', 'risk developing severe illness', 'antibiotics effective preventing treating covid19', 'medicines therapies prevent cure covid19', 'vaccine drug treatment covid19', 'covid19 sars', 'wear mask protect', 'use dispose mask', 'long incubation period covid19', 'humans infected covid19 animal source', 'catch covid19 pet', 'long virus survive surfaces', 'safe receive package area covid19 reported', '', 'source causing covid19 known', 'human sarscov2 infections occur', 'covid19 airborne', 'pregnant women higher risk covid19', 'im pregnant protect covid19', 'pregn

**Bag of words Model**

In [11]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

question_orig=" Can i get covid19 through air ?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)

0  :  a
1  :  coronavirus
2  :  is
3  :  what
4  :  covid19
5  :  are
6  :  of
7  :  symptoms
8  :  the
9  :  does
10  :  how
11  :  spread
12  :  air
13  :  be
14  :  can
15  :  causes
16  :  that
17  :  through
18  :  transmitted
19  :  virus
20  :  caught
21  :  from
22  :  has
23  :  no
24  :  person
25  :  who
26  :  catch
27  :  disease
28  :  feces
29  :  i
30  :  someone
31  :  with
32  :  14
33  :  andprotection
34  :  areas
35  :  days
36  :  do
37  :  for
38  :  have
39  :  in
40  :  measures
41  :  myself
42  :  or
43  :  past
44  :  persons
45  :  prevent
46  :  protect
47  :  recently
48  :  spreading
49  :  to
50  :  visited
51  :  where
52  :  protection
53  :  am
54  :  likely
55  :  about
56  :  should
57  :  worry
58  :  at
59  :  developing
60  :  illness
61  :  risk
62  :  severe
63  :  antibiotics
64  :  effective
65  :  preventing
66  :  treating
67  :  any
68  :  cure
69  :  medicines
70  :  therapies
71  :  there
72  :  drug
73  :  treatment
74  :  vaccine
75  

In [12]:

import sklearn
from sklearn.metrics.pairwise import cosine_similarity;

def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        
    
print( retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences) )

0 0.24253562503633297 what is a coronavirus
1 0.9761870601839527 what is covid19
2 0.9970544855015815 what are the symptoms of covid19
3 1.0 how does covid19 spread
4 1.0 can the virus that causes covid19 be transmitted through the air
5 0.24253562503633297 can covid19 be caught from a person who has no symptoms
6 1.0 can i catch covid19 from the feces of someone with the disease
7 0.9761870601839527 what can i do to protect myself andprotection measures for persons who are in or have recently visited past 14 days areas where covid19 is spreading prevent the spread of disease
8 0.9761870601839527 protection measures for persons who are in or have recently visited past 14 days areas where covid19 is spreading
9 1.0 how likely am i to catch covid19
10 1.0 how likely am i to catch covid19
11 1.0 should i worry about covid19
12 0.9761870601839527 who is at risk of developing severe illness
13 1.0 are antibiotics effective in preventing or treating the covid19
14 1.0 are there any medicines

**word 2vec**

In [14]:
from gensim.models import Word2Vec 
import gensim.downloader as api

v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved word2v model")

w2vec_embedding_size=len(v2w_model['computer']);

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved glove model


In [16]:
def getWordVec(word,model):
        samp=model['computer']
        vec=[0]*len(samp)
        try:
                vec=model[word]
        except:
                vec=[0]*len(samp)
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel)
        vec=numpy.array([0]*len(samp))
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel))
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [17]:
sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model))

question_embedding=getPhraseEmbedding(question,v2w_model)

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences)

0 0.014743232701589958 coronavirus
1 0.0 covid19
2 0.09510770487268358 symptoms covid19
3 0.21974295663091584 covid19 spread
4 0.42716614194711905 virus causes covid19 transmitted air
5 0.22947491576239842 covid19 caught person symptoms
6 0.26729394856137184 catch covid19 feces disease
7 0.3126140907618052 protect andprotection measures persons recently visited past 14 days areas covid19 spreading prevent spread disease
8 0.2642899717547245 protection measures persons recently visited past 14 days areas covid19 spreading
9 0.34921703672407806 likely catch covid19
10 0.34921703672407806 likely catch covid19
11 0.2348425295788565 worry covid19
12 0.16020520341425687 risk developing severe illness
13 0.17159759083457218 antibiotics effective preventing treating covid19
14 0.17381523529751147 medicines therapies prevent cure covid19
15 0.10840807935893483 vaccine drug treatment covid19
16 0.14178550930542885 covid19 sars
17 0.2765302987392187 wear mask protect
18 0.29182739390706486 use di